In [15]:
'''
generate the gamma for 1,5,10,50,100
'''

from data_import import data_import
from frame_func import frame_func
from window_func import window_func
from mel_func import mel_func
from DCT_func import DCT_func
from process_bar import ShowProcess

#default 
import pickle
import numpy as np
#loadmat
from scipy.io import loadmat,savemat

t_feed=10 #feed time
t_frame=20 #frame time
sample_rate=16000
fs=sample_rate/1000 #sample_rate of each ms
L_value=np.int(fs*t_frame)
NFFT=512
nfilt=22

audio_path = "D:\\LAB\\workspace\\lab\\patRecDat\\forStudents\\timit\\test"
#audio_path = "/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/timit/test"

dataset=data_import(audio_path,0)  #samples is a dictionary of 171 persons
print(len(dataset.keys()))
for gamma in [1,5,10,50,100]:
    feature_all_set={}
    print("feature engineering start")
    process_bar=ShowProcess(len(dataset.keys()))
    for name in dataset.keys():
        #print(name)
        process_bar.show_process()
        #print("make the feature of "+ name)
        single_data=dataset.get(name,'no such file name') # samples of one person
        features_set=[]
        for samples in single_data:
            if name in ['yuxin','qianqian','shanqi']: #custom voice has dimension error
                samples=samples[:,0]
            else:
                pass
            frames=frame_func(samples,gamma)  #frames is a list with length of 320*frames
            #print(len(frames))
            window_frames=window_func(frames) #using hanning window

            freq_frames=mel_func(window_frames)# 

            features=DCT_func(freq_frames)# features: 15*frames

            #features=features.ravel() # need to be reshape when import again

            features_set.append(features)

        feature_all_set.setdefault(name)

        feature_all_set[name]=features_set

    save_path="D:\\LAB\\lab\\task_2_version_3\\features_"+str(gamma)+".txt"
    f = open(save_path,'wb')
    pickle.dump(feature_all_set,f)
    f.close()
    #savemat(save_path,feature_all_set)  
    process_bar.close()
print("feature extraction compleleted")

170
feature engineering start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
feature engineering start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
feature engineering start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
feature engineering start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
feature engineering start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
feature extraction compleleted


In [16]:
'''
for gamma 1
'''
from process_bar import ShowProcess
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_3\\features_1.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
confusion_matrix=np.zeros((num_samples,num_samples))
correct_sum=0
false_sum=0
for cross_num in range(10):
	#process_bar.show_process()
	train_file_set=[]
	test_file_set=[]
	correct_num=0
	false_num=0
	'''
	Split the test and train set
	'''
	for name in features.keys():
		whole_set=features.get(name,'no such file').copy()
		test_file=whole_set[cross_num]
		train_num=list(range(10))
		train_num.remove(cross_num)
		test_file_set.append(test_file)
        #name_set.append(name)
		train_set=[]
		for num in train_num:
			train_set.append(whole_set[num])
		train_set=np.concatenate(train_set,axis=1)
		train_file_set.append(train_set)
	'''
	Start modeling and identification
	'''
	print("crossvalidation "+str(cross_num+1)+" start")
	process_bar_2=ShowProcess(num_samples)
	scores_set=np.zeros((num_samples,num_samples))
	for index_2 in range(num_samples):
		process_bar_2.show_process()
		b_train=train_file_set[index_2]
		gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=1,weights_init=ubm_weights,\
							means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
		gmm.fit(b_train.T)
		for index_1 in range(num_samples):
			#print("now test set "+str(index_1)+" is testing "+str(index_2))
			b_test=np.array(test_file_set[index_1])
			scores_set[index_1,index_2]=gmm.score(b_test.T)

	'''
	Calculate the detection rate
	'''
	for index in range(num_samples):
		test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
		if index == test_index:
			correct_num +=1
			correct_sum +=1
			confusion_matrix[index,index] +=1
		else:
			false_num +=1
			false_sum +=1
			confusion_matrix[index,test_index] +=1
			print("error ! True: "+str(name_set[index])+" False: "+str(name_set[test_index]))
		#print("time cost %5.1f second"%((time.time()-start)/60))

	process_bar_2.close()

	detection_rate=correct_num/(correct_num+false_num)
	print("crossvalidation "+str(cross_num+1)+" compeleted")
	print("cost time %5.1f minute"%((time.time()-start)/60))
	detection_rate=correct_num/(false_num+correct_num)
	detection_rate_set.append(detection_rate)
	print("the crossval "+str(cross_num)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))

#save_path="D:\\LAB\\lab\\task_2_version_4\\confusion_matrix.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
#f = open(save_path,'wb')
#features=pickle.dump(confusion_matrix,f)
#f.close()




crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 1 compeleted
cost time   4.1 minute
the crossval 0 detection_rate is 1.0
crossvalidation 2 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 2 compeleted
cost time   7.6 minute
the crossval 1 detection_rate is 1.0
crossvalidation 3 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 3 compeleted
cost time  11.6 minute
the crossval 2 detection_rate is 1.0
crossvalidation 4 start
error ! True: mbwm0 False: mklt0>>>>>>>>>>>>>>>>>>>]100.00%
error ! True: fjmg0 False: fadg0
error ! True: mgrt0 False: mtmr0
done
crossvalidation 4 compeleted
cost time  15.3 minute
the crossval 3 detection_rate is 0.9823529411764705
crossvalidation 5 start
error ! True: mrrk0 False: mcrc0>>>>>>>>>>>>>>>>>>>]100.00%
done
crossvalidation 5 compeleted
cost time  19.2 minute
the crossval 4 detection_rate is 0.9941176470588236
crossvalidation 6 start
error ! True: fcmh0 False: fd

In [17]:
'''
for gamma 5
'''
from process_bar import ShowProcess
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_3\\features_5.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
confusion_matrix=np.zeros((num_samples,num_samples))
correct_sum=0
false_sum=0
for cross_num in range(10):
	#process_bar.show_process()
	train_file_set=[]
	test_file_set=[]
	correct_num=0
	false_num=0
	'''
	Split the test and train set
	'''
	for name in features.keys():
		whole_set=features.get(name,'no such file').copy()
		test_file=whole_set[cross_num]
		train_num=list(range(10))
		train_num.remove(cross_num)
		test_file_set.append(test_file)
        #name_set.append(name)
		train_set=[]
		for num in train_num:
			train_set.append(whole_set[num])
		train_set=np.concatenate(train_set,axis=1)
		train_file_set.append(train_set)
	'''
	Start modeling and identification
	'''
	print("crossvalidation "+str(cross_num+1)+" start")
	process_bar_2=ShowProcess(num_samples)
	scores_set=np.zeros((num_samples,num_samples))
	for index_2 in range(num_samples):
		process_bar_2.show_process()
		b_train=train_file_set[index_2]
		gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=1,weights_init=ubm_weights,\
							means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
		gmm.fit(b_train.T)
		for index_1 in range(num_samples):
			#print("now test set "+str(index_1)+" is testing "+str(index_2))
			b_test=np.array(test_file_set[index_1])
			scores_set[index_1,index_2]=gmm.score(b_test.T)

	'''
	Calculate the detection rate
	'''
	for index in range(num_samples):
		test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
		if index == test_index:
			correct_num +=1
			correct_sum +=1
			confusion_matrix[index,index] +=1
		else:
			false_num +=1
			false_sum +=1
			confusion_matrix[index,test_index] +=1
			print("error ! True: "+str(name_set[index])+" False: "+str(name_set[test_index]))
		#print("time cost %5.1f second"%((time.time()-start)/60))

	process_bar_2.close()

	detection_rate=correct_num/(correct_num+false_num)
	print("crossvalidation "+str(cross_num+1)+" compeleted")
	print("cost time %5.1f minute"%((time.time()-start)/60))
	detection_rate=correct_num/(false_num+correct_num)
	detection_rate_set.append(detection_rate)
	print("the crossval "+str(cross_num)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))

#save_path="D:\\LAB\\lab\\task_2_version_4\\confusion_matrix.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
#f = open(save_path,'wb')
#features=pickle.dump(confusion_matrix,f)
#f.close()




crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 1 compeleted
cost time   4.1 minute
the crossval 0 detection_rate is 1.0
crossvalidation 2 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 2 compeleted
cost time   7.3 minute
the crossval 1 detection_rate is 1.0
crossvalidation 3 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 3 compeleted
cost time  10.9 minute
the crossval 2 detection_rate is 1.0
crossvalidation 4 start
error ! True: mbwm0 False: mklt0>>>>>>>>>>>>>>>>>>>]100.00%
error ! True: fjmg0 False: fadg0
error ! True: mgrt0 False: mtmr0
done
crossvalidation 4 compeleted
cost time  14.4 minute
the crossval 3 detection_rate is 0.9823529411764705
crossvalidation 5 start
error ! True: mrrk0 False: mcrc0>>>>>>>>>>>>>>>>>>>]100.00%
done
crossvalidation 5 compeleted
cost time  18.4 minute
the crossval 4 detection_rate is 0.9941176470588236
crossvalidation 6 start
error ! True: fdrw0 False: fs

In [18]:
'''
for gamma 10
'''
from process_bar import ShowProcess
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_3\\features_10.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
confusion_matrix=np.zeros((num_samples,num_samples))
correct_sum=0
false_sum=0
for cross_num in range(10):
	#process_bar.show_process()
	train_file_set=[]
	test_file_set=[]
	correct_num=0
	false_num=0
	'''
	Split the test and train set
	'''
	for name in features.keys():
		whole_set=features.get(name,'no such file').copy()
		test_file=whole_set[cross_num]
		train_num=list(range(10))
		train_num.remove(cross_num)
		test_file_set.append(test_file)
        #name_set.append(name)
		train_set=[]
		for num in train_num:
			train_set.append(whole_set[num])
		train_set=np.concatenate(train_set,axis=1)
		train_file_set.append(train_set)
	'''
	Start modeling and identification
	'''
	print("crossvalidation "+str(cross_num+1)+" start")
	process_bar_2=ShowProcess(num_samples)
	scores_set=np.zeros((num_samples,num_samples))
	for index_2 in range(num_samples):
		process_bar_2.show_process()
		b_train=train_file_set[index_2]
		gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=1,weights_init=ubm_weights,\
							means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
		gmm.fit(b_train.T)
		for index_1 in range(num_samples):
			#print("now test set "+str(index_1)+" is testing "+str(index_2))
			b_test=np.array(test_file_set[index_1])
			scores_set[index_1,index_2]=gmm.score(b_test.T)

	'''
	Calculate the detection rate
	'''
	for index in range(num_samples):
		test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
		if index == test_index:
			correct_num +=1
			correct_sum +=1
			confusion_matrix[index,index] +=1
		else:
			false_num +=1
			false_sum +=1
			confusion_matrix[index,test_index] +=1
			print("error ! True: "+str(name_set[index])+" False: "+str(name_set[test_index]))
		#print("time cost %5.1f second"%((time.time()-start)/60))

	process_bar_2.close()

	detection_rate=correct_num/(correct_num+false_num)
	print("crossvalidation "+str(cross_num+1)+" compeleted")
	print("cost time %5.1f minute"%((time.time()-start)/60))
	detection_rate=correct_num/(false_num+correct_num)
	detection_rate_set.append(detection_rate)
	print("the crossval "+str(cross_num)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))

#save_path="D:\\LAB\\lab\\task_2_version_4\\confusion_matrix.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
#f = open(save_path,'wb')
#features=pickle.dump(confusion_matrix,f)
#f.close()




crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 1 compeleted
cost time   3.8 minute
the crossval 0 detection_rate is 1.0
crossvalidation 2 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 2 compeleted
cost time   7.0 minute
the crossval 1 detection_rate is 1.0
crossvalidation 3 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 3 compeleted
cost time  10.7 minute
the crossval 2 detection_rate is 1.0
crossvalidation 4 start
error ! True: mbwm0 False: mklt0>>>>>>>>>>>>>>>>>>>]100.00%
error ! True: fjmg0 False: fadg0
error ! True: mgrt0 False: mtmr0
done
crossvalidation 4 compeleted
cost time  14.3 minute
the crossval 3 detection_rate is 0.9823529411764705
crossvalidation 5 start
error ! True: mrrk0 False: mcrc0>>>>>>>>>>>>>>>>>>>]100.00%
done
crossvalidation 5 compeleted
cost time  18.6 minute
the crossval 4 detection_rate is 0.9941176470588236
crossvalidation 6 start
error ! True: fdrw0 False: fs

In [19]:
'''
for gamma 50
'''
from process_bar import ShowProcess
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_3\\features_50.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
confusion_matrix=np.zeros((num_samples,num_samples))
correct_sum=0
false_sum=0
for cross_num in range(10):
	#process_bar.show_process()
	train_file_set=[]
	test_file_set=[]
	correct_num=0
	false_num=0
	'''
	Split the test and train set
	'''
	for name in features.keys():
		whole_set=features.get(name,'no such file').copy()
		test_file=whole_set[cross_num]
		train_num=list(range(10))
		train_num.remove(cross_num)
		test_file_set.append(test_file)
        #name_set.append(name)
		train_set=[]
		for num in train_num:
			train_set.append(whole_set[num])
		train_set=np.concatenate(train_set,axis=1)
		train_file_set.append(train_set)
	'''
	Start modeling and identification
	'''
	print("crossvalidation "+str(cross_num+1)+" start")
	process_bar_2=ShowProcess(num_samples)
	scores_set=np.zeros((num_samples,num_samples))
	for index_2 in range(num_samples):
		process_bar_2.show_process()
		b_train=train_file_set[index_2]
		gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=1,weights_init=ubm_weights,\
							means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
		gmm.fit(b_train.T)
		for index_1 in range(num_samples):
			#print("now test set "+str(index_1)+" is testing "+str(index_2))
			b_test=np.array(test_file_set[index_1])
			scores_set[index_1,index_2]=gmm.score(b_test.T)

	'''
	Calculate the detection rate
	'''
	for index in range(num_samples):
		test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
		if index == test_index:
			correct_num +=1
			correct_sum +=1
			confusion_matrix[index,index] +=1
		else:
			false_num +=1
			false_sum +=1
			confusion_matrix[index,test_index] +=1
			print("error ! True: "+str(name_set[index])+" False: "+str(name_set[test_index]))
		#print("time cost %5.1f second"%((time.time()-start)/60))

	process_bar_2.close()

	detection_rate=correct_num/(correct_num+false_num)
	print("crossvalidation "+str(cross_num+1)+" compeleted")
	print("cost time %5.1f minute"%((time.time()-start)/60))
	detection_rate=correct_num/(false_num+correct_num)
	detection_rate_set.append(detection_rate)
	print("the crossval "+str(cross_num)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))

#save_path="D:\\LAB\\lab\\task_2_version_4\\confusion_matrix.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
#f = open(save_path,'wb')
#features=pickle.dump(confusion_matrix,f)
#f.close()




crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 1 compeleted
cost time   4.0 minute
the crossval 0 detection_rate is 1.0
crossvalidation 2 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 2 compeleted
cost time   7.5 minute
the crossval 1 detection_rate is 1.0
crossvalidation 3 start
error ! True: msfh1 False: mjdh0>>>>>>>>>>>>>>>>>>>]100.00%
done
crossvalidation 3 compeleted
cost time  11.4 minute
the crossval 2 detection_rate is 0.9941176470588236
crossvalidation 4 start
error ! True: mtmr0 False: mcsh0>>>>>>>>>>>>>>>>>>>]100.00%
error ! True: mbwm0 False: mklt0
error ! True: fjmg0 False: fadg0
error ! True: mgrt0 False: mtmr0
done
crossvalidation 4 compeleted
cost time  15.0 minute
the crossval 3 detection_rate is 0.9764705882352941
crossvalidation 5 start
error ! True: mrrk0 False: mcrc0>>>>>>>>>>>>>>>>>>>]100.00%
done
crossvalidation 5 compeleted
cost time  18.9 minute
the crossval 4 detection_rate is 0.9941176470588236


In [20]:
'''
for gamma 100
'''
from process_bar import ShowProcess
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_3\\features_100.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
confusion_matrix=np.zeros((num_samples,num_samples))
correct_sum=0
false_sum=0
for cross_num in range(10):
	#process_bar.show_process()
	train_file_set=[]
	test_file_set=[]
	correct_num=0
	false_num=0
	'''
	Split the test and train set
	'''
	for name in features.keys():
		whole_set=features.get(name,'no such file').copy()
		test_file=whole_set[cross_num]
		train_num=list(range(10))
		train_num.remove(cross_num)
		test_file_set.append(test_file)
        #name_set.append(name)
		train_set=[]
		for num in train_num:
			train_set.append(whole_set[num])
		train_set=np.concatenate(train_set,axis=1)
		train_file_set.append(train_set)
	'''
	Start modeling and identification
	'''
	print("crossvalidation "+str(cross_num+1)+" start")
	process_bar_2=ShowProcess(num_samples)
	scores_set=np.zeros((num_samples,num_samples))
	for index_2 in range(num_samples):
		process_bar_2.show_process()
		b_train=train_file_set[index_2]
		gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=1,weights_init=ubm_weights,\
							means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
		gmm.fit(b_train.T)
		for index_1 in range(num_samples):
			#print("now test set "+str(index_1)+" is testing "+str(index_2))
			b_test=np.array(test_file_set[index_1])
			scores_set[index_1,index_2]=gmm.score(b_test.T)

	'''
	Calculate the detection rate
	'''
	for index in range(num_samples):
		test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
		if index == test_index:
			correct_num +=1
			correct_sum +=1
			confusion_matrix[index,index] +=1
		else:
			false_num +=1
			false_sum +=1
			confusion_matrix[index,test_index] +=1
			print("error ! True: "+str(name_set[index])+" False: "+str(name_set[test_index]))
		#print("time cost %5.1f second"%((time.time()-start)/60))

	process_bar_2.close()

	detection_rate=correct_num/(correct_num+false_num)
	print("crossvalidation "+str(cross_num+1)+" compeleted")
	print("cost time %5.1f minute"%((time.time()-start)/60))
	detection_rate=correct_num/(false_num+correct_num)
	detection_rate_set.append(detection_rate)
	print("the crossval "+str(cross_num)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))

#save_path="D:\\LAB\\lab\\task_2_version_4\\confusion_matrix.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_3/features.txt"
#f = open(save_path,'wb')
#features=pickle.dump(confusion_matrix,f)
#f.close()




crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 1 compeleted
cost time   3.6 minute
the crossval 0 detection_rate is 1.0
crossvalidation 2 start
done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
crossvalidation 2 compeleted
cost time   6.7 minute
the crossval 1 detection_rate is 1.0
crossvalidation 3 start
error ! True: msfh1 False: mjdh0>>>>>>>>>>>>>>>>>>>]100.00%
done
crossvalidation 3 compeleted
cost time  10.4 minute
the crossval 2 detection_rate is 0.9941176470588236
crossvalidation 4 start
error ! True: mtmr0 False: mcsh0>>>>>>>>>>>>>>>>>>>]100.00%
error ! True: fjmg0 False: fadg0
error ! True: mgrt0 False: mtmr0
error ! True: qianqian False: yuxin
done
crossvalidation 4 compeleted
cost time  13.8 minute
the crossval 3 detection_rate is 0.9764705882352941
crossvalidation 5 start
error ! True: mmjr0 False: mbjk0>>>>>>>>>>>>>>>>>>>]100.00%
error ! True: mrrk0 False: mcrc0
done
crossvalidation 5 compeleted
cost time  17.6 minute
the crossval 4 d